# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,la plata,-34.9215,-57.9545,16.68,76,17,2.56,AR,1691607735
1,1,talnakh,69.4865,88.3972,16.23,64,100,3.30,RU,1691607856
2,2,adamstown,-25.0660,-130.1015,19.07,73,19,9.76,PN,1691607856
3,3,lihue,21.9789,-159.3672,29.58,71,75,9.77,US,1691607857
4,4,hadibu,12.6500,54.0333,25.69,80,74,7.11,YE,1691607857


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"   )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#ideal_city_df = city_data_df[city_data_df['Humidity'] <= 50] 
#ideal_city2_df = ideal_city_df[ideal_city_df['Max Temp'] <= 25] # don't like hot or muggy, fine with cold and clouds > so Minnesota works
ideal_city_df = city_data_df[(city_data_df['Humidity'] <= 50) & (city_data_df['Max Temp'] <= 25)]
# Drop any rows with null values
ideal_city_df.dropna

# Display sample data
#ideal_city_df.count()  # for tracking the reduction
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,altai,46.3722,96.2583,14.99,50,1,2.62,MN,1691607862
44,44,isafjordur,66.0755,-23.1240,13.02,50,0,6.69,IS,1691607866
55,55,port hedland,-20.3167,118.5667,17.03,45,1,3.60,AU,1691607764
79,79,udachny,66.4167,112.4000,16.36,47,100,3.34,RU,1691607880
82,82,yellowknife,62.4560,-114.3525,20.29,45,75,9.77,CA,1691607687


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df.drop(["Max Temp", "Cloudiness", "Wind Speed", "Date"], axis = 1, inplace=True)
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,la plata,AR,-34.9215,-57.9545,
1,talnakh,RU,69.4865,88.3972,
2,adamstown,PN,-25.0660,-130.1015,
3,lihue,US,21.9789,-159.3672,
4,hadibu,YE,12.6500,54.0333,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la plata - nearest hotel: Hotel Catedral
talnakh - nearest hotel: Талнах
adamstown - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
hadibu - nearest hotel: No hotel found
mugumu - nearest hotel: Mama hotel
papatowai - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
hermanus - nearest hotel: Aloe guest house
port-aux-francais - nearest hotel: Keravel
kogalym - nearest hotel: Сибирь
pacific grove - nearest hotel: Pacific Grove Inn
puerto santa - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
invercargill - nearest hotel: The Grand
hamilton - nearest hotel: North Vista Manor
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
grytviken - nearest hotel: No hotel found
murun-kuren - nearest hotel: No hotel found
vorgashor - nearest hotel: София
ust'-barguzin - nearest hotel: Байкальский бриз
edinburgh of the seven seas - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel 

,City,Country,Lat,Lng,Hotel Name
0,la plata,AR,-34.9215,-57.9545,Hotel Catedral
1,talnakh,RU,69.4865,88.3972,Талнах
2,adamstown,PN,-25.0660,-130.1015,No hotel found
3,lihue,US,21.9789,-159.3672,Kauai Palms
4,hadibu,YE,12.6500,54.0333,No hotel found
...,...,...,...,...,...
566,contamana,PE,-7.3333,-75.0167,No hotel found
567,okha,RU,53.5739,142.9478,Сфера
568,bandarbeyla,SO,9.4942,50.8122,No hotel found
569,anna regina,GY,7.2644,-58.5077,Jaigobin Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["Country", "Hotel Name"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)